In [1]:
project_id = 'elife-data-pipeline'
output_dataset = 'de_dev'
output_table_prefix = 'data_science_'

In [2]:
import logging
import sys

import data_science_pipeline.configure_warnings

from data_science_pipeline.sql import get_sql
from data_science_pipeline.utils.bq import run_query_and_save_to_table, get_client
from data_science_pipeline.utils.jupyter import printmd, to_markdown_sql

In [3]:
destination_table_name = ''.join([output_table_prefix, 'Manuscript_Version_Matching_Reviewing_Editor_Profile'])

In [4]:
logging.basicConfig(level='INFO', stream=sys.stdout)

In [5]:
print('processing Manuscript_Version_Matching_Reviewing_Editor_Profile')
_sql = get_sql('Manuscript_Version_Matching_Reviewing_Editor_Profile.sql').format(
    project=project_id,
    dataset=output_dataset
)
printmd(to_markdown_sql(_sql))
run_query_and_save_to_table(
    client=get_client(project_id=project_id),
    query=_sql,
    destination_dataset=output_dataset,
    destination_table_name=destination_table_name
)
print('done')

processing Manuscript_Version_Matching_Reviewing_Editor_Profile


> ```sql
> WITH t_full_submission_version_id_including_null AS (
>   -- add NULL to also have one entry for the general editor search ("no manuscript selected")
>   SELECT NULL AS version_id
>   
>   UNION ALL
> 
>   SELECT version_id
>   FROM `elife-data-pipeline.de_dev.v_manuscript_version_last_editor_assigned_timestamp` AS manuscript_version
>   WHERE manuscript_version.overall_stage = 'Full Submission'
>   AND TIMESTAMP_DIFF(CURRENT_TIMESTAMP(), manuscript_version.stages[SAFE_OFFSET(0)].stage_timestamp, DAY) <= 90
>   AND (
>     TIMESTAMP_DIFF(CURRENT_TIMESTAMP(), manuscript_version.stages[SAFE_OFFSET(0)].stage_timestamp, DAY) <= 30
>     OR NOT EXISTS (SELECT 1 FROM UNNEST(manuscript_version.stages) WHERE stage_name = 'Reviewing Editor Assigned')
>   )
>   AND NOT is_withdrawn
>   AND NOT is_deleted
> ),
> 
> t_recommendations AS (
>   SELECT
>     Editor.*,
>     manuscript_version.manuscript_id AS Manuscript_ID,
>     manuscript_version.version_id AS Version_ID,
>     manuscript_version.long_manuscript_identifier AS Long_Manuscript_Identifier,
>     COALESCE(
>       CONCAT(manuscript_version.long_manuscript_identifier, ' - ', manuscript_version.manuscript_title),
>       '~ No Manuscript Selected ~'
>     ) AS Display_Manuscript_Identifier,
>     manuscript_version.overall_stage AS Overall_Stage,
>     manuscript_version.manuscript_title AS Manuscript_Title,
>     manuscript_version.abstract AS Abstract,
>     ARRAY_TO_STRING(
>       ARRAY(
>         SELECT keyword
>         FROM UNNEST(matching_keywords)
>       ),
>       ', '
>     ) AS Matching_Keywords_CSV,
>     editor_recommendation.score AS Matching_Score,
>     (
>       MAX(editor_recommendation.score) OVER(
>         PARTITION BY manuscript_version.version_id
>       ) IS NOT NULL
>     ) AS Has_Recommendation,
>     manuscript_version.stages[SAFE_OFFSET(0)].stage_name AS Last_Stage_Name,
>     (
>       SELECT stage_timestamp
>       FROM UNNEST(stages)
>       WHERE stage_name = 'Reviewing Editor Assigned'
>       ORDER BY stage_timestamp DESC
>       LIMIT 1
>     ) AS Reviewing_Editor_Assigned_Timestamp
>   FROM `elife-data-pipeline.de_dev.mv_Editorial_Editor_Profile` AS Editor
>   CROSS JOIN t_full_submission_version_id_including_null AS version_ids
>   LEFT JOIN `elife-data-pipeline.de_dev.v_manuscript_version_last_editor_assigned_timestamp` AS manuscript_version
>     ON manuscript_version.version_id = version_ids.version_id
>   LEFT JOIN `elife-data-pipeline.de_dev.data_science_reviewing_editor_recommendation` AS editor_recommendation
>     ON editor_recommendation.person_id = Editor.Person_ID
>     AND editor_recommendation.version_id = version_ids.version_id
>   WHERE Editor.Role_Name = 'Reviewing Editor'
> )
> 
> SELECT
>   *,
>   IF(
>     Version_ID IS NULL,
>     Display_Manuscript_Identifier,
>     CONCAT(
>       Long_Manuscript_Identifier,
>       IF(Reviewing_Editor_Assigned_Timestamp IS NOT NULL, ' (RE assigned)', ''),
>       IF(
>         NOT Has_Recommendation,
>         ' (no recommendation yet)',
>         ''
>       ),
>       ' - ',
>       Manuscript_Title
>     )
>   ) AS Display_Manuscript_Identifier_With_State
> FROM t_recommendations
> ```

INFO:data_science_pipeline.utils.bq:ran query and saved to: de_dev.data_science_Manuscript_Version_Matching_Reviewing_Editor_Profile, total rows: 370512, took: 23.207s
done
